https://www.analyticsvidhya.com/blog/2021/12/text-classification-of-news-articles/

ปัญหาของข้อมูลชุดนี้
- ไม่รู้ว่า Label คือ column ใด
- ต้องเปลี่ยน Keywords จาก List เป็น String
- ลบเครื่องหมายต่างๆ

In [1]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

#### import data from cleansing

In [25]:
df = pd.read_csv('../data/cleaned/cleaned-data.csv')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4480 entries, 0 to 4479
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      4480 non-null   object
 1   label     4480 non-null   object
 2   headline  4480 non-null   object
 3   snippet   4472 non-null   object
 4   url       4480 non-null   object
dtypes: object(5)
memory usage: 175.1+ KB


#### Values checking

In [27]:
def find_string(a, b, c):
    '''
    ฟังชั่นสำหรับหา String จาก column อย่างเฉพาะเจาะจง
    a = DataFrame Column
    b = 
    c = ตัวอักษรที่จะค้นหา
    Return = DataFrame
    '''
    string = []
    index = []
    for i in df[a]:
        string.append(i.find(c))
    for j in df.index:
        index.append(j)
    df[b] = string
    df['index'] = index
    return df[['index',a,b]].loc[df[b] > -1]

find_string('url', 'ind', 'politics')

,index,url,ind
9,9,https://www.nytimes.com/2019/12/31/us/politics...,38
33,33,https://www.nytimes.com/2020/01/01/us/politics...,38
38,38,https://www.nytimes.com/2020/01/01/us/politics...,38
68,68,https://www.nytimes.com/2020/01/01/us/politics...,38
85,85,https://www.nytimes.com/2020/01/01/us/politics...,38
...,...,...,...
4461,4461,https://www.nytimes.com/2020/01/31/us/politics...,38
4467,4467,https://www.nytimes.com/2020/01/31/us/politics...,38
4469,4469,https://www.nytimes.com/2020/01/31/us/politics...,38
4474,4474,https://www.nytimes.com/2020/01/31/us/politics...,38


In [28]:
def count_values(a):
    '''
    a = ชื่อ column
    ฟั่งชั่นนับจำนวน Label ทั้งหมด
    Return = DataFrame 
    '''
    label = []
    u_names = []
    count = []
    for i in df[a]:
        label.append(i)
    for names in label:
        if names not in u_names:
            names = str(names)
            u_names.append(names)
    for num in u_names:
        count.append(label.count(num))
    return pd.DataFrame({'Label': u_names, 'Numbers': count})

Num_values = count_values('label')
Num_values[['Label', 'Numbers']].loc[Num_values['Numbers'] > 200]

,Label,Numbers
0,us,287
1,opinion,503
5,world,482
6,politics,494
8,arts,346
9,business,297
18,sports,226


#### Word Cloud